# Web Scraping & Data Cleaning

### Workflow

**Data Acquisition**
1) Scraped 50 pages of album data from metacritic, sorted by release date.\
2) Access Spotify's API to get album URIs for each album title that we scraped (checking to be sure it is a match and we didn't just return a different album by the same name)\
3) Use those album URIs to query Spotify API again for track level information on each album, including track names, durations, and track URIs\
4) Use the individual track URIs to query one more time and return "audio features" for each track on each album.\
5) Get artist links for each artist in our dataset, and then use those to scrape each artist's Spotify landing page for monthly listener totals 

In [1]:
#Imports needed for web scraping
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

#In case I need to make a quick visualization to investigate any 
#features I'm adding
import matplotlib.pyplot as plt
import seaborn as sns


### Data Acquisition

We will be scraping our data from metacritic's list of [new releases](https://www.metacritic.com/browse/albums/release-date/available/date?view=condensed). The next four cells show the workflow for scraping 50 pages of data using BeautifulSoup and placing the results in a Pandas dataframe

In [122]:
#Define the base URL we will be using to web scrape

base_url = 'https://www.metacritic.com/browse/albums/release-date/available/date?view=condensed'
user_agent = {'User-agent':'Mac Chrome'}

In [123]:
#Empty lists for the attributes we will be scraping from metacritic

critic_reviews_links = []
titles = []
critic_scores = []
user_scores = []
artists = []
descriptions = []
dates_raw = []

In [124]:
#Scrape data for each album: title, artist, critic score, user score, description, release date
for i in range(0,50):
    #Select page of results
    url_ext = f'&page={i}'
    url = base_url+url_ext
    response = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text,'lxml')
    
    #Isolate table contents with album info
    tables = soup.findAll('table')
    table_rows = []
    for table in tables:
        table_rows.append(table.findAll('tr'))

    #Add info for each album
    for group in table_rows:
        for row in group:
            critic_reviews_links.append(row.find('a')['href'])
            titles.append(row.findAll('td')[1].find('h3').text)
            critic_scores.append(row.find('div').text)
            user_scores.append(row.findAll('td')[1].find('div',class_='user').text)
            artists.append(' '.join(row.findAll('td')[1].find('div').text.strip().split(' ')[1:]))
            descriptions.append(row.findAll('td')[1].find('p').text)
            if row.findAll('td')[1].find('span').text:
                dates_raw.append(row.findAll('td')[1].find('span').text)
            else:
                dates_raw.append('Empty')

In [125]:
#Create pandas dataframe with scraped data

album_dict = {'title': titles, 'artist':artists, 'critic_score':critic_scores, \
             'user_score':user_scores, 'description':descriptions, \
              'date_raw':dates_raw, 'critic_reviews_link':critic_reviews_links}
album_df = pd.DataFrame(data=album_dict)

Note: I did some data cleaning steps, such as the one below, while I was still in the "data acquisition" phase.  This was in order to remove rows that I knew wouldn't be used, and cut down on run time for later steps

In [127]:
#Drop compilation albums with "Various Artists" that we won't be able to get additional features for
album_df = album_df[album_df['artist'] != 'Various Artists'].reset_index() 

In [129]:
#Remove things like "[Deluxe Edition]" or "[EP]"from end of titles so that we can use the album titles
#to search for album data via Spotify's API

new_title_list = []

for title in album_df.title:
    if title.find('[') != -1:
        new_title_list.append(title.split(' [')[0])
    else:
        new_title_list.append(title)

album_df['title'] = new_title_list

In [130]:
#Imports that will be needed to access Spotify's API
import spotipy
import json
import urllib.parse
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

In order to use Spotify's API, you need to follow Spotipy's client credentials flow, which allows you to authenticate your requests.  To do this, you will need to have a Spotify account and register your app at [My Dashboard](https://developer.spotify.com/dashboard/) to get the credentials needed.

In [132]:
#Set client credentials for accessing Spotify's API.  Note: removed my personal credentials
#for privacy purposes

client_id = ''  
client_secret = '' 
username = ''  
scope = 'user-library-read' 
redirect_uri = 'https://www.spotify.com/us/'

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, \
                                                    client_id=client_id, \
                                                    client_secret=client_secret, \
                                                    redirect_uri=redirect_uri,\
                                                    username=username
                                                   )
                         )

In [2]:
# Search Spotify to get album URIs for each title in dataset 
# (only if there is a match), and add to album dataframe

# Total tracks for the album is also given in the search results, 
# so create column for that as well

tracks = []
uris = []

for ind, name in enumerate(album_df.title):
    results = spotify.search(q='album:' + name, type='album', limit = 30)
    
    if results['albums']['items'] == []:
        tracks.append('unknown')
        uris.append('unknown')
    
    for i, result in enumerate(results['albums']['items']):
        if result['artists'][0]['name'] == album_df.loc[ind, 'artist']:
            tracks.append(results['albums']['items'][i]['total_tracks'])
            uris.append(results['albums']['items'][i]['uri'])
            break
        if result == results['albums']['items'][-1]:
            tracks.append('unknown')
            uris.append('unknown')

album_df['num_tracks'] = tracks
album_df['uri'] = uris

In [156]:
#Remove results where I couldn't retrieve URI/number of tracks
album_df = album_df[album_df['num_tracks'] != 'unknown'].reset_index(drop=True,inplace=True)

In [161]:
#For each album in dataset, use URI to get information on album tracks, 
#including durations, names, and track URIs

duration_ms_list = []
explicit_list = []
name_list = []
uri_list = []


for link in album_df.uri:
    duration_ms = []
    explicit = []
    name = []
    uri = []
    #Get a list of track by track information for a single album
    album_tracks = spotify.album_tracks(link) 
    
    #Loop through that album's track to get a list of features
    for track in album_tracks['items']:
        duration_ms.append(track['duration_ms'])
        explicit.append(track['explicit'])
        name.append(track['name'])
        uri.append(track['uri'])
        
    #Add that album's feature lists to overall lists that will become dataframe columns
    duration_ms_list.append(duration_ms)
    explicit_list.append(explicit)
    name_list.append(name)
    uri_list.append(uri)
    
album_df['track_durations'] = duration_ms_list
album_df['explicit_tracks'] = explicit_list
album_df['track_names'] = name_list
album_df['track_uris'] = uri_list


In [164]:
#Use track URIs to get individual audio features for each song on an album

#For each album, get a list of Spotify's audio features track by track

danceability_list = []
energy_list = []
loudness_list = []
speechiness_list = []
acousticness_list = []
instrumentalness_list = []
liveness_list = []
valence_list = []
tempo_list = []

for i, uri_list in enumerate(album_df['track_uris']):
    #create a list of audio features for each album
    danceability = []
    energy = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    for track_uri in uri_list:
        #Access spotify's audio features for each track on the album
        audio_features = spotify.audio_features(track_uri)[0]
        if audio_features == None:
            danceability.append('Unknown')
            energy.append('Unknown')
            loudness.append('Unknown')
            speechiness.append('Unknown')
            acousticness.append('Unknown')
            instrumentalness.append('Unknown')
            liveness.append('Unknown')
            valence.append('Unknown')
            tempo.append('Unknown')
        
        #Add the audio features for each track to the album list created above
        else: 
            danceability.append(audio_features['danceability'])
            energy.append(audio_features['energy'])
            loudness.append(audio_features['loudness'])
            speechiness.append(audio_features['speechiness'])
            acousticness.append(audio_features['acousticness'])
            instrumentalness.append(audio_features['instrumentalness'])
            liveness.append(audio_features['liveness'])
            valence.append(audio_features['valence'])
            tempo.append(audio_features['tempo'])
        
    danceability_list.append(danceability)
    energy_list.append(energy)
    loudness_list.append(loudness)
    speechiness_list.append(speechiness)
    acousticness_list.append(acousticness)
    instrumentalness_list.append(instrumentalness)
    liveness_list.append(liveness)
    valence_list.append(valence)
    tempo_list.append(tempo)

In [165]:
album_df['danceability'] = danceability_list
album_df['energy'] = energy_list
album_df['loudness'] = loudness_list
album_df['speechiness'] = speechiness_list
album_df['acousticness'] = acousticness_list
album_df['instrumentalness'] = instrumentalness_list
album_df['liveness'] = liveness_list
album_df['valence'] = valence_list
album_df['tempo'] = tempo_list

In [ ]:
#Get artist links from Spotify API to use for scraping each individual artist's page
artist_link_list = []
for i, uri in enumerate(album_df['uri']):
    print(i)
    album = spotify.album(uri)
    artist_link_list.append(album['artists'][0]['id'])

album_df['artist_link'] = artist_link_list

# Scrape artist pages and get artist's monthly listeners to use as a metric for popularity
base_url = 'https://open.spotify.com/artist/'
monthly_listener_list = []
for ind, link in enumerate(album_df['artist_link']):
    url = base_url + link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    monthly_listeners = int(soup.findAll('h3', class_='insights__column__number')[0].text.replace(',',''))
    monthly_listener_list.append(monthly_listeners)

album_df['artist_monthly_listeners'] = monthly_listener_list

In [6]:
#Sort by highest monthly listeners as a sense check
album_df.sort_values('artist_monthly_listeners', ascending=False).head(7)

,Unnamed: 0,title,artist,critic_score,user_score,description,date_raw,critic_reviews_link,num_tracks,uri,...,acousticness_avg,instrumentalness_avg,liveness_avg,valence_avg,tempo_avg,release_date,release_month,day_of_week_release,artist_link,artist_monthly_listeners
2111,2111,Starboy,The Weeknd,67,7.9,The third full-length studio release for the C...,"November 25, 2016",/music/starboy/the-weeknd/critic-reviews,18,spotify:album:2ODvWsOgouMbaA5xf0RkJe,...,0.182,0.011,0.164,0.469,130.640,2016-11-25,11,4,1Xyo4u8uXC1ZmMpatF05PJ,62695747
385,385,After Hours,The Weeknd,80,8.6,The fourth full-length studio release for the ...,"March 20, 2020",/music/after-hours/the-weeknd/critic-reviews,14,spotify:album:4yP0hdKOZPNshxUOjY0cZj,...,0.122,0.012,0.194,0.251,121.553,2020-03-20,3,4,1Xyo4u8uXC1ZmMpatF05PJ,62695747
1872,1872,More Life,Drake,79,6.8,"The ""playlist"" project for the Canadian hip-ho...","March 18, 2017",/music/more-life-mixtape/drake/critic-reviews,22,spotify:album:1lXY618HWkwYKJWBRYR4MK,...,0.198,0.025,0.263,0.372,119.888,2017-03-18,3,5,3TVXtAsR1Inumwj472S9r4,61702881
659,659,Care Package,Drake,77,6.5,The 17-track collection of tracks not included...,"August 2, 2019",/music/care-package/drake/critic-reviews,17,spotify:album:7dqpveMVcWgbzqYrOdkFTD,...,0.406,0.001,0.117,0.375,122.710,2019-08-02,8,4,3TVXtAsR1Inumwj472S9r4,61702881
273,273,Dark Lane Demo Tapes,Drake,61,4.7,"The collection of demos, singles and unrelease...","May 1, 2020",/music/dark-lane-demo-tapes-mixtape/drake/crit...,14,spotify:album:6OQ9gBfg5EXeNAEwGSs6jK,...,0.316,0.000,0.255,0.482,114.240,2020-05-01,5,4,3TVXtAsR1Inumwj472S9r4,61702881
498,498,Changes,Justin Bieber,57,5.3,The first full-length studio release in nearly...,"February 14, 2020",/music/changes/justin-bieber/critic-reviews,17,spotify:album:63iWSELt9V1kV6RSMxN7Ii,...,0.405,0.000,0.127,0.649,130.168,2020-02-14,2,4,1uNFoZAHBGtllmzznpCI3s,57048275
728,728,Oasis,J Balvin,79,6.6,The debut full-length release for the collabor...,"June 28, 2019",/music/oasis/j-balvin/critic-reviews,8,spotify:album:6ylFfzx32ICw4L1A7YWNLN,...,0.091,0.000,0.182,0.709,126.141,2019-06-28,6,4,1vyhD5VmyZ7KMfW5gqLgo5,55744791


We see names like The Weeknd, Drake, and Justin Bieber at the top, so it likes like our scraping worked

### Data Cleaning

In [166]:
#Find titles with tracks containing unknown audio features
for index, feature_list in enumerate(album_df['danceability']):
    if 'Unknown' in feature_list:
        print(index, album_df.loc[index, 'title'])

95 Western Swing & Waltzes and Other Punchy Songs
2094 Jackie


In [168]:
#Only two albums contained tracks with unknown audio features, so just drop those albums from dataframe
album_df.drop([95, 2094], axis=0, inplace=True)

#Reset index 
album_df.reset_index(drop=True, inplace=True)

In [ ]:
#Get total album length in and average track length for each album from "duration_ms" lists

album_length_list = []
avg_track_length_list = []

for ind, length_list in enumerate(album_df.track_durations):
    #Convert individual track lengths from ms to s
    seconds_array = np.array(length_list)/1000
    
    #Set album length and avg track length
    album_length = "%.3f" % (np.sum(seconds_array))
    avg_track_length = "%.3f" % (np.sum(seconds_array)/len(seconds_array))
    
    album_length_list.append(album_length)
    avg_track_length_list.append(avg_track_length)

album_df['album_length_seconds'] = album_length_list
album_df['avg_track_length_seconds'] = avg_track_length_list

In [ ]:
#Calculate what percentage of each album's tracks are explicit, and add column to dataframe

percent_explicit_list = []

for ind, explicit_list in enumerate(album_df.explicit_tracks):
    total_explicit = sum(explicit_list)
    percent_explicit = "%.3f" % (total_explicit / len(explicit_list) * 100)
    percent_explicit_list.append(percent_explicit)
album_df['percent_explicit'] = percent_explicit_list

In [172]:
#For audio features where each track has a value, get features that are the mean of each track

danceability_avg_list = []
energy_avg_list = []
loudness_avg_list = []
speechiness_avg_list = []
acousticness_avg_list = []
instrumentalness_avg_list = []
liveness_avg_list = []
valence_avg_list = []
tempo_avg_list = []


for ind in album_df.index:
    
    #Each entry was one long string with single quotes at the ends, so slice from [1:-1]
    #to drop the single quotes, split at the commas to get a list of the numbers in string format, 
    #and convert each to a float.
    danceability_avg = "%.3f" % (sum(album_df.loc[ind, 'danceability'])/len(album_df.loc[ind, 'danceability']))
    

    #Then calculate the average across all tracks for each album
    danceability_avg_list.append(danceability_avg)
   
    #Repeat process for remaining features
    energy = album_df.loc[ind, 'energy']
    energy_avg_list.append("%.3f" % (sum(energy)/len(energy)))
    
    loudness = album_df.loc[ind, 'loudness']
    loudness_avg_list.append("%.3f" % (sum(loudness)/len(loudness)))
    
    speechiness = album_df.loc[ind, 'speechiness']
    speechiness_avg_list.append("%.3f" % (sum(speechiness)/len(speechiness)))
    
    acousticness = album_df.loc[ind, 'acousticness']
    acousticness_avg_list.append("%.3f" % (sum(acousticness)/len(acousticness)))
    
    instrumentalness = album_df.loc[ind, 'instrumentalness']
    instrumentalness_avg_list.append("%.3f" % (sum(instrumentalness)/len(instrumentalness)))
    
    liveness = album_df.loc[ind, 'liveness']
    liveness_avg_list.append("%.3f" % (sum(liveness)/len(liveness)))
    
    valence = album_df.loc[ind, 'valence']
    valence_avg_list.append("%.3f" % (sum(valence)/len(valence)))
    
    tempo = album_df.loc[ind, 'tempo']
    tempo_avg_list.append("%.3f" % (sum(tempo)/len(tempo)))
    
album_df['energy_avg'] = energy_avg_list
album_df['loudness_avg'] = loudness_avg_list
album_df['danceability_avg'] = danceability_avg_list
album_df['speechiness_avg'] = speechiness_avg_list
album_df['acousticness_avg'] = acousticness_avg_list
album_df['instrumentalness_avg'] = instrumentalness_avg_list
album_df['liveness_avg'] = liveness_avg_list
album_df['valence_avg'] = valence_avg_list
album_df['tempo_avg'] = tempo_avg_list

In [173]:
#Convert the raw dates into datetime, and then add columns for release month and day of week b

album_df['release_date'] = pd.to_datetime(album_df['date_raw'], format='%B %d, %Y')
album_df['release_month'] = album_df['release_date'].dt.month
album_df['day_of_week_release'] = album_df['release_date'].dt.weekday

In [207]:
# Scrape artist pages and get artist's monthly listeners to use as a metric for popularity
base_url = 'https://open.spotify.com/artist/'
monthly_listener_list = []
for ind, link in enumerate(album_df['artist_link'].tolist()):
    url = base_url + link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    monthly_listeners = int(soup.findAll('h3', class_='insights__column__number')[0].text.replace(',',''))
    monthly_listener_list.append(monthly_listeners)

album_df['artist_monthly_listeners'] = monthly_listener_list

In [210]:
#Save dataframe to use later
album_df.to_csv('more_data_fixed.csv')